<a href="https://colab.research.google.com/github/100477706/Proyecto_Aprendizaje/blob/main/predicciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>